<a href="https://colab.research.google.com/github/adityaras/2048-Solver-using-TD-Learning/blob/master/rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/RL

/content/drive/MyDrive/RL


In [3]:
import numpy as np 
import pandas as pd 
import os
import random
import numpy
import copy
import math
from itertools import permutations 

In [5]:
def print_board(tiles):
    print(" ---------------")
    for i in range(4):
        print("| ",end="")
        for j in range(4):
            if tiles[i][j]==0:
                print(" "+" | ",end="")
            else:
                print(str(tiles[i][j])+" | ",end="")
        print("")
        print(" ---------------")

def new_tile(tiles):
    available_cells=[]
    for i in range(4):
        for j in range(4):
            if tiles[i][j]==0:
                available_cells.append([i,j])
    pos=random.randint(0,len(available_cells)-1)
    val=numpy.random.choice([2,4],p=[0.9,0.1])
    return available_cells[pos],val

def check_game_over(tiles):
    for i in range(4):
        for j in range(4):
            if tiles[i][j]==0:
                return False
    for i in range(4):
        for j in range(3):
            if tiles[i][j]==tiles[i][j+1]:
                return False
    for j in range(4):
        for i in range(3):
            if tiles[i][j]==tiles[i+1][j]:
                return False
    return True

def take_action(tiles,action):
    temp=copy.deepcopy(tiles)
    update=0
    if action==1:
        for col in range(4):
            for row in range(1,4):
                if tiles[row][col]!=0:
                    i=row-1
                    f=0
                    while(i>=0 and tiles[i][col]==0):
                        f=1
                        i-=1
                    if f==1:
                        tiles[i+1][col]=tiles[row][col]
                        tiles[row][col]=0
            for row in range(3):
                if tiles[row][col]==tiles[row+1][col]:
                    tiles[row][col]+=tiles[row][col]
                    update+=tiles[row][col]
                    for i in range(row+1,3):
                        tiles[i][col]=tiles[i+1][col]
                    tiles[3][col]=0
    elif action==2:
        for col in range(4):
            for row in range(2,-1,-1):
                if tiles[row][col]!=0:
                    i=row+1
                    f=0
                    while(i<4 and tiles[i][col]==0):
                        i+=1
                        f=1
                    if f==1:
                        tiles[i-1][col]=tiles[row][col]
                        tiles[row][col]=0
            for row in range(3,0,-1):
                if tiles[row][col]==tiles[row-1][col]:
                    tiles[row][col]+=tiles[row][col]
                    update+=tiles[row][col]
                    for i in range(row-1,0,-1):
                        tiles[i][col]=tiles[i-1][col]
                    tiles[0][col]=0
    elif action==3:
        for row in range(4):
            for col in range(1,4):
                if tiles[row][col]!=0:
                    f=0
                    i=col-1
                    while(i>=0 and tiles[row][i]==0):
                        i-=1
                        f=1
                    if f==1:
                        tiles[row][i+1]=tiles[row][col]
                        tiles[row][col]=0
            for col in range(3):
                if tiles[row][col]==tiles[row][col+1]:
                    tiles[row][col]+=tiles[row][col]
                    update+=tiles[row][col]
                    for i in range(col+1,3):
                        tiles[row][i]=tiles[row][i+1]
                    tiles[row][3]=0
    elif action==4:
        for row in range(4):
            for col in range(2,-1,-1):
                if tiles[row][col]!=0:
                    f=0
                    i=col+1
                    while(i<4 and tiles[row][i]==0):
                        f=1
                        i+=1
                    if f==1:
                        tiles[row][i-1]=tiles[row][col]
                        tiles[row][col]=0
            for col in range(3,0,-1):
                if tiles[row][col]==tiles[row][col-1]:
                    tiles[row][col]+=tiles[row][col]
                    update+=tiles[row][col]
                    for i in range(col-1,0,-1):
                        tiles[row][i]=tiles[row][i-1]
                    tiles[row][0]=0
    game_over=check_game_over(tiles)
    if game_over:
        return tiles,update,game_over
    not_changed=check_same(tiles,temp)
    while not_changed:
        print("this action caused no change!")
        print("Choose Action : 1.Up 2.Down 3.Left 4.Right")
        action=int(input())
        tiles,update,game_over=take_action(tiles,action)
        not_changed=check_same(tiles,temp)
    return tiles,update,game_over

def check_same(x,y):
    for i in range(4):
        for j in range(4):
            if x[i][j]!=y[i][j]:
                return False
    return True


In [6]:
def custom_log(value):
  if value==0:
    return str(hex(int(0)))
  else:
    return str(hex(int(math.log(value,2))))

In [7]:
ind={}
count=0
for i in range(16):
  for j in range(16):
    for k in range(16):
      for l in range(16):
            st=str(hex(i))+str(hex(j))+str(hex(k))+str(hex(l))
            ind[st]=count
            count+=1

In [8]:
def line(grid):
  ax=[]
  grid_rot=grid
  for i in range(3):
    txt=""
    txt+=custom_log(grid_rot[0][i])
    txt+=custom_log(grid_rot[1][i])
    txt+=custom_log(grid_rot[2][i])
    txt+=custom_log(grid_rot[3][i])
    ax.append(txt)
  for j in range(1,3):
    grid_rot=np.rot90(grid,k=j)
    for i in range(3):
      txt=""
      txt+=custom_log(grid_rot[0][i])
      txt+=custom_log(grid_rot[1][i])
      txt+=custom_log(grid_rot[2][i])
      txt+=custom_log(grid_rot[3][i])
      ax.append(txt)
  pos=np.array([np.array([0]*1) for i in range(len(ind.keys()))])
  for i in ax:
    pos[ind[i]][0]=1
  return pos

In [9]:
def count_pair_of_mergable_tiles(tiles):
    x=0
    for i in range(4):
        for j in range(3):
            if tiles[i][j]==tiles[i][j+1]:
                x+=1
    for j in range(4):
        for i in range(3):
            if tiles[i][j]==tiles[i+1][j]:
                x+=1
    return x

In [10]:
def count_empty_tiles(tiles):
    x=0
    for i in range(4):
        for j in range(4):
            if tiles[i][j]==0:
                x+=1
    return x

In [11]:
def count_distinct_tiles(tiles):
    tileset={}
    for i in range(4):
        for j in range(4):
            if tiles[i][j] not in tileset:
                tileset[tiles[i][j]]=1
    return len(tileset.keys())

In [13]:
def count_layered(grid): #relies on the fact that an empty tile is NOT represented as 0. 1 would be prefered for empty tile. 
    count = 0
    for col in range(4):
        if((grid[0][col]==2*grid[1][col]) and grid[0][col]!=0):
            count+=1
        if((grid[3][col]==2*grid[2][col]) and grid[3][col]!=0):
            count+=1
        for row in range(1, 3):
            if((grid[row][col]==2*grid[row+1][col] or grid[row][col]==2*grid[row-1][col]) and  grid[row][col]!=0):
                count+=1
    return(count)

In [14]:
def calculate_v(state,weights):
  phi = feature_vec(state)
  v = phi.dot(weights)
  return v

In [15]:
def count_large_tiles(grid):
  count=0
  for i in grid:
    for j in i:
      if j>=1024:
        count+=1
  return count

In [25]:
def feature_vec(state):
  feat=[]
  feat.append(np.array([count_layered(state)]))
  feat.append(np.array([count_distinct_tiles(state)]))
  feat.append(np.array([count_empty_tiles(state)]))
  feat.append(np.array([count_pair_of_mergable_tiles(state)]))
  feat.append(np.array([count_large_tiles(state)]))
  feat=np.concatenate((np.array(feat),np.array(line(state))))
  return np.array(feat).reshape((1,5+len(ind.keys())))

In [17]:
def generate_game():
  tiles=np.array([np.array([0]*4) for i in range(4)])
  pos,val=new_tile(tiles)
  tiles[pos[0]][pos[1]]=val
  pos,val=new_tile(tiles)
  tiles[pos[0]][pos[1]]=val
  return tiles

In [18]:
def count_rect(grid):
  count = 0
  for row in range(3):
      for col in range(1, 3):
        if(grid[row][col]!=0 and grid[row][col-1]!=0 and grid[row][col+1]!=0 and grid[row+1][col]!=0 and grid[row+1][col-1]!=0 and grid[row+1][col+1]!=0):
          count+=1
  for col in range(3):
      for row in range(1, 3):
        if(grid[row][col]!=0 and grid[row-1][col]!=0 and grid[row+1][col]!=0 and grid[row][col+1]!=0 and grid[row+1][col+1]!=0 and grid[row-1][col+1]!=0):
          count+=1
  return count

In [19]:
def next_move(tiles,action):
  temp=copy.deepcopy(tiles)
  update=0
  if action==1:
      for col in range(4):
          for row in range(1,4):
              if tiles[row][col]!=0:
                  i=row-1
                  f=0
                  while(i>=0 and tiles[i][col]==0):
                      f=1
                      i-=1
                  if f==1:
                      tiles[i+1][col]=tiles[row][col]
                      tiles[row][col]=0
          for row in range(3):
              if tiles[row][col]==tiles[row+1][col]:
                  tiles[row][col]+=tiles[row][col]
                  update+=tiles[row][col]
                  for i in range(row+1,3):
                      tiles[i][col]=tiles[i+1][col]
                  tiles[3][col]=0
  elif action==2:
      for col in range(4):
          for row in range(2,-1,-1):
              if tiles[row][col]!=0:
                  i=row+1
                  f=0
                  while(i<4 and tiles[i][col]==0):
                      i+=1
                      f=1
                  if f==1:
                      tiles[i-1][col]=tiles[row][col]
                      tiles[row][col]=0
          for row in range(3,0,-1):
              if tiles[row][col]==tiles[row-1][col]:
                  tiles[row][col]+=tiles[row][col]
                  update+=tiles[row][col]
                  for i in range(row-1,0,-1):
                      tiles[i][col]=tiles[i-1][col]
                  tiles[0][col]=0
  elif action==3:
      for row in range(4):
          for col in range(1,4):
              if tiles[row][col]!=0:
                  f=0
                  i=col-1
                  while(i>=0 and tiles[row][i]==0):
                      i-=1
                      f=1
                  if f==1:
                      tiles[row][i+1]=tiles[row][col]
                      tiles[row][col]=0
          for col in range(3):
              if tiles[row][col]==tiles[row][col+1]:
                  tiles[row][col]+=tiles[row][col]
                  update+=tiles[row][col]
                  for i in range(col+1,3):
                      tiles[row][i]=tiles[row][i+1]
                  tiles[row][3]=0
  elif action==4:
      for row in range(4):
          for col in range(2,-1,-1):
              if tiles[row][col]!=0:
                  f=0
                  i=col+1
                  while(i<4 and tiles[row][i]==0):
                      f=1
                      i+=1
                  if f==1:
                      tiles[row][i-1]=tiles[row][col]
                      tiles[row][col]=0
          for col in range(3,0,-1):
              if tiles[row][col]==tiles[row][col-1]:
                  tiles[row][col]+=tiles[row][col]
                  update+=tiles[row][col]
                  for i in range(col-1,0,-1):
                      tiles[row][i]=tiles[row][i-1]
                  tiles[row][0]=0
  game_over=check_game_over(tiles)
  not_changed=check_same(tiles,temp)
  if game_over or not_changed:
      return tiles,0,True
  pos,val=new_tile(tiles)
  tiles[pos[0]][pos[1]]=val
  return tiles, update,False

In [26]:
def single_stage_td():
  weights1 = np.zeros((5+len(ind.keys()),1))
  weights2 = np.zeros((5+len(ind.keys()),1))
  weights3 = np.zeros((5+len(ind.keys()),1))
  actions = np.array([1,2,3,4])
  weights = copy.deepcopy(weights1) 
  dic={1:"u",2:"d",3:"l",4:"r"}
  alpha = 0.00001
  epsilon=1
  cumm_reward=[]
  for i in range(100):
    state = generate_game() 
    timesteps = 1
    weights = copy.deepcopy(weights1)
    max_value=0
    tot_reward=0
    while True:
      if max_value==math.pow(2,10):
        weights1 = copy.deepcopy(weights)
        weights = copy.deepcopy(weights2)
      if max_value==math.pow(2,13):
        weights2 = copy.deepcopy(weights)
        weights = copy.deepcopy(weights3)
      epsilon = 1 / timesteps
      timesteps += 1
      state_values = {}
      action_selection = []
      for a in actions:
        temp_state = copy.deepcopy(state)
        next_state,r,checker = next_move(temp_state,a)
        if not checker:
          action_selection.append(calculate_v(next_state,weights)[0][0])
        else:
          action_selection.append(float('-inf'))

      if np.max(action_selection) == float('-inf'):
        print(i,state) 
        t = np.amax(state)
        if max_value < t:
          max_value = t
        print("\n")     
        break
      p=random.random()
      if p<(1-epsilon):
        selected_action = np.argmax(action_selection) + 1  
      else:
        selected_action = random.randint(1,4)
      next_state, reward, checker = next_move(state,selected_action)
      cumm_reward.append(reward)
      phi = feature_vec(state)
      if str(state) not in state_values:
        state_values[str(state)] = 0
      state_values[str(state)] = phi.dot(weights)

      weights = weights + alpha * (reward + np.max(action_selection) - state_values[str(state)]) * np.transpose(phi) 
      state = next_state

  return weights1, weights2, weights3, cumm_reward


In [ ]:
a,b,c,wt = single_stage_td()

0 [[  2   8 128   2]
 [  4  64  32  16]
 [ 32  16 256   4]
 [  4   8  64  32]]




In [ ]:
np.save("w_ntup_1.sav",a)
np.save("w_ntup_2.sav",b)
np.save("w_ntup_3.sav",c)
np.save("rewards_ntup.sav",wt)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(num=None, figsize=(15,7.5), dpi=80, facecolor='w', edgecolor='k')
plt.plot(wt)
plt.ylabel('Cummulative Rewards')
plt.xlabel('Runs')
plt.show()